# ETL PROCESSES

# Developing the ETL process for each of the tables.

In [1]:
# Imports
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
# Connectiong to the database
conn = psycopg2.connect("host=127.0.0.1 dbname=data_eng user=postgres")
cur = conn.cursor()

OperationalError: connection to server at "127.0.0.1", port 5432 failed: fe_sendauth: no password supplied


In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))

    return all_files

# Process song_data

In [4]:
song_files = get_files('./data/song_data/')
filepath = song_files[0]

In [5]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARDNS031187B9924F0,32.67828,-83.22295,Georgia,Tim Wilson,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,186.48771,2005
